<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/final_VSM_bai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install underthesea

In [193]:
from underthesea import word_tokenize
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [194]:
FRIEND_PRE_DATA = list()
VOCAB_SIZE = 0

In [195]:
path = '/content/drive/MyDrive/NLP/chatbotdata/bạn bè.txt'
with open(path) as frienddata:
  i = 0
  for data in frienddata:
    sent = data.split("__eou__")[:2]
    if len(sent) == 2:
      anq = list()
      for subsent in sent:
        anq.append(subsent)
      FRIEND_PRE_DATA.append(anq)

In [196]:
FRIEND_PRE_DATA[0]

['Thích đánh lộn không? ', ' Ngon nhà vô ']

In [197]:
df = pd.DataFrame(FRIEND_PRE_DATA, columns=["Question", "Answer"])

In [198]:
df['Merge'] = df['Question'] + df["Answer"]

In [199]:
df.head(10)

,Question,Answer,Merge
0,Thích đánh lộn không?,Ngon nhà vô,Thích đánh lộn không? Ngon nhà vô
1,Solo yasua không,Chấp lun 2 mạng đầu,Solo yasua không Chấp lun 2 mạng đầu
2,Mai đi picnic không?,Mai bận học rồi,Mai đi picnic không? Mai bận học rồi
3,Mai học ca mấy vậy?,Mai học ca 3,Mai học ca mấy vậy? Mai học ca 3
4,Còn tiền không?,Còn chết liền,Còn tiền không? Còn chết liền
5,Mấy giờ rồi nhỉ?,11:00 rồi,Mấy giờ rồi nhỉ? 11:00 rồi
6,Bao lâu rồi chưa về quê,1 tháng rồi,Bao lâu rồi chưa về quê 1 tháng rồi
7,Bạn có crush chưa,Mình có rồi,Bạn có crush chưa Mình có rồi
8,Crush bao lâu rồi,Khoảng 10 năm,Crush bao lâu rồi Khoảng 10 năm
9,Crush có Bồ chưa?,Có rồi huhu,Crush có Bồ chưa? Có rồi huhu


In [200]:
document_token = [ word_tokenize(sent.lower()) for sent in df['Merge']] 

In [201]:
df["Tokenize"] = document_token

In [202]:
df.head(10)

,Question,Answer,Merge,Tokenize
0,Thích đánh lộn không?,Ngon nhà vô,Thích đánh lộn không? Ngon nhà vô,"[thích, đánh lộn, không, ?, ngon, nhà, vô]"
1,Solo yasua không,Chấp lun 2 mạng đầu,Solo yasua không Chấp lun 2 mạng đầu,"[solo, yasua, không, chấp, lun, 2, mạng, đầu]"
2,Mai đi picnic không?,Mai bận học rồi,Mai đi picnic không? Mai bận học rồi,"[mai, đi, picnic, không, ?, mai, bận, học, rồi]"
3,Mai học ca mấy vậy?,Mai học ca 3,Mai học ca mấy vậy? Mai học ca 3,"[mai, học, ca, mấy, vậy, ?, mai, học, ca, 3]"
4,Còn tiền không?,Còn chết liền,Còn tiền không? Còn chết liền,"[còn, tiền, không, ?, còn, chết, liền]"
5,Mấy giờ rồi nhỉ?,11:00 rồi,Mấy giờ rồi nhỉ? 11:00 rồi,"[mấy, giờ, rồi, nhỉ, ?, 11, :, 00, rồi]"
6,Bao lâu rồi chưa về quê,1 tháng rồi,Bao lâu rồi chưa về quê 1 tháng rồi,"[bao lâu, rồi, chưa, về, quê, 1, tháng, rồi]"
7,Bạn có crush chưa,Mình có rồi,Bạn có crush chưa Mình có rồi,"[bạn, có, crush, chưa, mình, có, rồi]"
8,Crush bao lâu rồi,Khoảng 10 năm,Crush bao lâu rồi Khoảng 10 năm,"[crush, bao lâu, rồi, khoảng, 10, năm]"
9,Crush có Bồ chưa?,Có rồi huhu,Crush có Bồ chưa? Có rồi huhu,"[crush, có, bồ, chưa, ?, có, rồi, huhu]"


In [203]:
w2v_model = Word2Vec(df['Tokenize'], size = 300, min_count = 2, workers = 3)

In [204]:
w2v_model.wv.most_similar("khoảng")

[('nhau', 0.9995696544647217),
 (',', 0.9995543956756592),
 ('rồi', 0.9995535612106323),
 ('.', 0.9995489120483398),
 ('?', 0.9995474815368652),
 ('người', 0.999547004699707),
 ('với', 0.9995461702346802),
 ('đi', 0.9995430111885071),
 ('hay', 0.9995427131652832),
 ('sao', 0.9995424151420593)]

In [205]:
print('Vocabulary size:', len(w2v_model.wv.vocab))

Vocabulary size: 598


In [206]:
w2v_model.wv.word_vec("crush").shape

(300,)

In [207]:
embedding_each_sent = list()
for sent_tokenize in df['Tokenize']:
  embedding = list()
  for word in sent_tokenize:
    if word not in w2v_model.wv.vocab:
      #300 dimension is the same as size of output embedding word2vec
      embedding.append(np.random.rand(300))  
    else:
      embedding.append(w2v_model.wv.word_vec(word))
  embedding = np.mean(embedding, axis = 0)
  embedding_each_sent.append(embedding)



In [208]:
np.array(embedding_each_sent).shape


(610, 300)

In [209]:
df['Embedding_vector'] = embedding_each_sent


In [210]:
df.head(30)


,Question,Answer,Merge,Tokenize,Embedding_vector
0,Thích đánh lộn không?,Ngon nhà vô,Thích đánh lộn không? Ngon nhà vô,"[thích, đánh lộn, không, ?, ngon, nhà, vô]","[-0.002419531425886325, 0.18361451128266476, 0..."
1,Solo yasua không,Chấp lun 2 mạng đầu,Solo yasua không Chấp lun 2 mạng đầu,"[solo, yasua, không, chấp, lun, 2, mạng, đầu]","[0.2177986643133716, 0.35278749777648666, 0.31..."
2,Mai đi picnic không?,Mai bận học rồi,Mai đi picnic không? Mai bận học rồi,"[mai, đi, picnic, không, ?, mai, bận, học, rồi]","[0.032702188216388074, 0.10600210901175876, 0...."
3,Mai học ca mấy vậy?,Mai học ca 3,Mai học ca mấy vậy? Mai học ca 3,"[mai, học, ca, mấy, vậy, ?, mai, học, ca, 3]","[-0.034582898, 0.06460502, -0.0091148, -0.0246..."
4,Còn tiền không?,Còn chết liền,Còn tiền không? Còn chết liền,"[còn, tiền, không, ?, còn, chết, liền]","[0.06924444304123388, 0.1280587441579126, 0.03..."
5,Mấy giờ rồi nhỉ?,11:00 rồi,Mấy giờ rồi nhỉ? 11:00 rồi,"[mấy, giờ, rồi, nhỉ, ?, 11, :, 00, rồi]","[0.009076516738299937, 0.07743546669266738, 0...."
6,Bao lâu rồi chưa về quê,1 tháng rồi,Bao lâu rồi chưa về quê 1 tháng rồi,"[bao lâu, rồi, chưa, về, quê, 1, tháng, rồi]","[-0.03717409, 0.06946865, -0.00984173, -0.0263..."
7,Bạn có crush chưa,Mình có rồi,Bạn có crush chưa Mình có rồi,"[bạn, có, crush, chưa, mình, có, rồi]","[-0.06998384, 0.13206232, -0.018603478, -0.050..."
8,Crush bao lâu rồi,Khoảng 10 năm,Crush bao lâu rồi Khoảng 10 năm,"[crush, bao lâu, rồi, khoảng, 10, năm]","[-0.031402793, 0.058421776, -0.007890015, -0.0..."
9,Crush có Bồ chưa?,Có rồi huhu,Crush có Bồ chưa? Có rồi huhu,"[crush, có, bồ, chưa, ?, có, rồi, huhu]","[-0.053210378, 0.10015448, -0.0144921085, -0.0..."


In [211]:
np.array([embedding]).shape

(1, 300)

In [212]:
df['Embedding_vector'][4].shape


(300,)

In [215]:
def ir():
  while(True):
    q = input()
    if q == "q":
      break
    
    q = q.lower()
    tokenize_q = word_tokenize(q)
    q_embedding = list()
    for word in tokenize_q:
      if word not in w2v_model.wv.vocab:
        #300 dimension is the same as size of output embedding word2vec
        q_embedding.append(np.random.rand(300))  
      else:
        q_embedding.append(w2v_model.wv.word_vec(word))
    q_embedding = np.mean(q_embedding, axis = 0)
  
    r = list()
    for i in (range(len(df['Embedding_vector']))):
      r.append([i, cosine_similarity(np.array([q_embedding]), np.array([df['Embedding_vector'][i]]))[0][0]])
    
    r= pd.DataFrame(r, columns = ["index", "cosine_similarity"])
    r = r.sort_values(by ="cosine_similarity", ascending= False)
    r = r.head(15)
    # print(r)
    a = df.iloc[r['index']]['Answer'].tolist()
    cosine = r['cosine_similarity'].tolist()
    
    print(pd.DataFrame( list(zip(a, cosine)) , columns = ["Answer", 'cosine_similarity']))
      

Gõ q để thoát


In [214]:
print("Gõ q để thoát")
ir()

bạn có crush ai không?
                                               Answer  cosine_similarity
0                                                 Có            0.999998
1    Mình đang crush một bạn, bạn có thể chỉ mình ...           0.999998
2     không, crush mình không biết mình thích bạn đó            0.999998
3                                          nhiều lắm            0.999998
4                                       có nhiều lắm            0.999998
5                                         Không ít à            0.999998
6                                         Không ít à            0.999998
7                                                 có            0.999998
8                                                 Có            0.999998
9                                       có nhiều lắm            0.999997
10                                                có            0.999997
11                                            Rất có            0.999997
12                          